In [ ]:
import ee
import geemap
ee.Initialize()

# Translate code from javascript to python

In [ ]:
javascript_code = """
// Sample the image
var sampledFeatures = ndviDifferenceImage.sample({
  scale: 1000 // Set scale to 1km to match MODIS resolution
});

// Convert feature collection back to image with NDVI values
var sampledImage = ee.Image().paint({
  featureCollection: sampledFeatures,
  color: 'NDVI', // Use the NDVI property from the features
  width: 1 // This will paint each feature as a 1 pixel wide point
});

// Now you can visualize `sampledImage` on the map


"""

# Translate
lines = geemap.js_snippet_to_py(
    javascript_code, add_new_cell=False,
    import_ee=True, import_geemap=True, show_map=True)
for line in lines:
    print(line.rstrip())

# Convert geemap to pandas df

In [ ]:
terraclimate = ee.ImageCollection('IDAHO_EPSCOR/TERRACLIMATE')
tmax = terraclimate.select('tmmx')

geometry = ee.Geometry.Point([77.57738128916243, 12.964758918835752])

def scale_image(image):
  return ee.Image(image).multiply(0.1)\
    .copyProperties(image, ['system:time_start'])

tmaxScaled = tmax.map(scale_image)

filtered = tmaxScaled.filter(ee.Filter.date('2019-01-01', '2020-01-01')) \
                     .filter(ee.Filter.bounds(geometry))

def extract_data(image):
    stats = image.reduceRegion(**{ 
        'reducer':ee.Reducer.mean(),
        'geometry':geometry,
        'scale':5000
    })
    properties = {
        'month': image.get('system:index'),
        'tmmx': stats.get('tmmx')
    }
    return ee.Feature(None, properties)

data = ee.FeatureCollection(filtered.map(extract_data))
print(data.size().getInfo())

from geemap import chart
options = {
    'title': 'Max Monthly Temperature at Bangalore', 
    'legend_location': 'top-right',
    'height': '500px',
    'ylabel': 'Temperature (C)',
    'xlabel': 'Date',
    'colors': ['blue']
}

df = geemap.ee_to_pandas(data)
print(df.head())